In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import parse
import json
import copy

In [3]:
def parse_transcript(filename):
    spaces_regex = re.compile("^(\s*).*")
    location_regex = re.compile("^\s*(INT\.|EXT\.)")
    line_list = []
    options = [0] * 100
    opt = []
    transcript = []

    text_file = open(filename, "r")
    lines = text_file.readlines()
    for l in lines:
        li = l.strip(' \n\t\r')
        if li != "":
            spmatch = spaces_regex.search(l)
            spaces_number = len(spmatch.group(1))
            options[spaces_number] += 1
            line_list.append((li, spaces_number))
    for i in range(0, 50):
        if options[i] >= 20:
            opt.append(i)
    print(opt)
    text_file.close()

    speaker = ""
    utterance = ""

    for l in lines:
        li = l.strip(' \n\t\r')
        if li != "":
            spmatch = spaces_regex.search(l)
            spaces_number = len(spmatch.group(1))
            if spaces_number == opt[2]:
                if utterance != "" and speaker != "":
                    transcript.append(
                        {'speaker': speaker, 'utterance': utterance.strip()})
                    utterance = ""
                speaker = re.sub(r'\([^()]*\)', '', li).strip(' \n\t\r')
            elif spaces_number == opt[1]:
                utterance += " " + li
            else:
                if utterance != "" and speaker != "":
                    transcript.append(
                        {'speaker': speaker, 'utterance': utterance.strip()})
                    utterance = ""
                    speaker = ""

    return transcript

In [4]:
SCRIPT_PATH = "./data/scripts/V-for-Vendetta.txt"
MOVIE_NAME = "V for Vendetta"
parsed_transcript = (parse_transcript(SCRIPT_PATH))

[10, 20, 30]


In [5]:
def make_line_dict(transcript):
    line_dict = dict()

    for i in np.arange(len(parsed_transcript)):
        speaker = parsed_transcript[i]['speaker']
        line = parsed_transcript[i]['utterance']
        if speaker in line_dict.keys():
            line_dict[speaker] += [line]
        else:
            line_dict[speaker] = [line]
            
    return line_dict

In [6]:
credits = pd.read_csv('./data/tmdb_5000_credits.csv')

In [7]:
movie_titles = np.array(credits["title"].tolist())
movie_num = np.where(movie_titles == "V for Vendetta")
movie_cast = np.array(credits["cast"].tolist())[movie_num][0]
line_dict = make_line_dict(parsed_transcript)

In [28]:
movie_char_list = json.loads(movie_cast)
gender_dict = dict()
movie_char_list_copy = copy.deepcopy(movie_char_list)


for speaker in line_dict.keys():
    speaker_dist = list()
    for d in movie_char_list_copy:
        if speaker.lower() == d['character'].lower() or speaker.lower() in ([x.lower() for x in d['character'].split(' ')]):            
            gender_dict[d['character']] = speaker, d['name'], d['gender']
            movie_char_list_copy.remove(d)
            break

In [33]:
ethnicities = pd.read_csv('./data/ethnicelebs.csv', header = None)
actor_names = ethnicities[0].tolist()
actor_ethnicities = ethnicities[1].tolist()


ethnicity_dict = dict()
            
for character in gender_dict.keys():
    speaker, name, gender = gender_dict[character]
    actor = '-'.join(name.lower().split(' '))
    try:
        actor_ethnicity = actor_ethnicities[actor_names.index(actor)]
        ethnicity_dict[character] = speaker, name, actor_ethnicity
    except ValueError:
        pass

gender_count = dict()
for character in gender_dict.keys():
    speaker, name, gender = gender_dict[character]
    num_lines = len(line_dict[speaker])
    if gender in gender_count.keys():
        gender_count[gender] += num_lines
    else:
        gender_count[gender] = num_lines
    
print(gender_count)

{0: 14, 1: 280, 2: 415}
